<a href="https://colab.research.google.com/github/SKD151105/Skin-Cancer-Classification/blob/main/model-inference_V2/Copy_of_vit_model_inference2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ======================================================
# Step 1: Imports
# ======================================================
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from PIL import Image
import os

# ======================================================
# Step 2: Load and preprocess data
# ======================================================
data = pd.read_csv('/content/drive/MyDrive/mini_proj_data/HAM10000/hmnist_28_28_RGB.csv')
X = data.drop(columns=['label']).values
y = data['label'].values

In [ ]:
X

In [ ]:
y

In [ ]:
# Reshape flattened pixels → 28x28x3
X = X.reshape(-1, 28, 28, 3).astype(np.uint8)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ======================================================
# Step 3: Prepare HuggingFace feature extractor
# ======================================================
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# ======================================================
# Step 4: Create Custom Dataset class
# ======================================================
class HMNISTDataset(Dataset):
    def __init__(self, images, labels, extractor):
        self.images = images
        self.labels = labels
        self.extractor = extractor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        inputs = self.extractor(images=img, return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return inputs

train_dataset = HMNISTDataset(X_train, y_train, feature_extractor)
val_dataset = HMNISTDataset(X_val, y_val, feature_extractor)


In [ ]:
# ======================================================
# Step 5: Load ViT model for classification
# ======================================================
num_classes = len(np.unique(y))
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k',
    num_labels=num_classes
)

In [ ]:
# ======================================================
# Step 7: Define metric computation
# ======================================================
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# ======================================================
# Step 8: Train the model
# ======================================================

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./vit_output',
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./vit_logs',
    load_best_model_at_end=True,
    report_to="none"  # 🚫 disables wandb / tensorboard logging cleanly
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()



In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
# ======================================================
# Step 9: Evaluate model
# ======================================================
preds = trainer.predict(val_dataset)
y_pred = np.argmax(preds.predictions, axis=1)
print("Validation Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

# ======================================================
# Step 10: Inference on all data & Save to CSV
# ======================================================
inference_dataset = HMNISTDataset(X, y, feature_extractor)
inference_loader = DataLoader(inference_dataset, batch_size=16)

model.eval()
results = []

with torch.no_grad():
    for i, batch in enumerate(inference_loader):
        inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        preds = probs.argmax(dim=-1).cpu().numpy()
        confs = probs.max(dim=-1).values.cpu().numpy()

        for j in range(len(preds)):
            results.append({
                'image_id': i * 16 + j,
                'true_label': int(batch['labels'][j]),
                'predicted_label': int(preds[j]),
                'confidence': float(confs[j])
            })

# Save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('/content/drive/MyDrive/mini_proj_data/vit_inference_results.csv', index=False)
print("Inference results saved to vit_inference_results.csv ✅")

In [ ]:
!cp -r /content/vit_output /content/drive/MyDrive/mini_proj_data/
